# Z2 lattice gauge theory

In [1]:
# The C2QA package is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import c2qa
import qiskit
import numpy as np
import scipy
import itertools
import c2qa.util as util
import c2qa.stateReadout as stateReadout
import matplotlib.pyplot as plt
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.providers.aer import AerSimulator
from qiskit import IBMQ
from collections import Counter

### Create a circuit with as many modes as qubits

In [3]:
numberofmodes=2
numberofqubitspermode=1
cutoff=2**numberofqubitspermode

qmr = c2qa.QumodeRegister(num_qumodes=numberofmodes, num_qubits_per_qumode=numberofqubitspermode)
qbr = qiskit.QuantumRegister(size=1)
cbr = qiskit.ClassicalRegister(size=1)
circuit = c2qa.CVCircuit(qmr, qbr, cbr)
# circuit = c2qa.CVCircuit(qmr)

### Initialise, e.g. share one boson between two cavities by initialising one cavity to Fock state 1 and the other to the vacuum.

In [4]:
diffstallmodes=[1,0]
for i in range(qmr.num_qumodes):
    circuit.cv_initialize(diffstallmodes[i], qmr[i])
word="diffstallmodes" #should correspond to the above line

In [5]:
def eiht(circuit, qma, qmb, qb, m, g, dt):
    circuit.cv_bs(2*dt, qmb, qma) # 2*dt because inside beamsplitter the phi is g/2
    # circuit.cv_r(dt*m, qma)
    # circuit.cv_r(dt*m, qmb)
    # circuit.rx(-2*g*dt, qb)
    return circuit

In [6]:
dt=1
m=1
g=1
N=5

circuit.h(qbr[0]) # Inititialises the qubit to a plus state (so that pauli Z flips it)
stateop, _ = c2qa.util.simulate(circuit)
util.stateread(stateop, qbr.size, numberofmodes, cutoff)
for i in range(N):
    print("dt+1")
    eiht(circuit, qmr[0], qmr[1], qbr[0], m, g, dt)
    stateop, _ = c2qa.util.simulate(circuit)
    util.stateread(stateop, qbr.size, numberofmodes, cutoff)

qumodes:  01  qubits:  0     with amplitude:  0.7071067811865476
qumodes:  01  qubits:  1     with amplitude:  0.7071067811865475
dt+1
qumodes:  01  qubits:  0     with amplitude:  0.3820514243700898
qumodes:  10  qubits:  0     with amplitude:  -0.595009839529386
qumodes:  01  qubits:  1     with amplitude:  0.38205142437008976
qumodes:  10  qubits:  1     with amplitude:  -0.5950098395293859
dt+1
qumodes:  01  qubits:  0     with amplitude:  -0.2942602500918142
qumodes:  10  qubits:  0     with amplitude:  -0.6429703766239181
qumodes:  01  qubits:  1     with amplitude:  -0.29426025009181417
qumodes:  10  qubits:  1     with amplitude:  -0.642970376623918
dt+1
qumodes:  01  qubits:  0     with amplitude:  -0.7000304076699753
qumodes:  10  qubits:  0     with amplitude:  -0.09978691466023233
qumodes:  01  qubits:  1     with amplitude:  -0.7000304076699752
qumodes:  10  qubits:  1     with amplitude:  -0.09978691466023232
dt+1
qumodes:  01  qubits:  0     with amplitude:  -0.462195836

/home/richard/anaconda3/envs/c2qa/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/home/richard/anaconda3/envs/c2qa/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
